# <font size=35 color=lightgreen>** Sentiment API **<font>
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [401]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import uvicorn
import sklearn
import fastapi
import joblib
import warnings
warnings.filterwarnings('ignore')

### <font size = 8 color="lightgreen">Importación del dataset <font>

In [402]:
# montar drive
# Nota: En un entorno de producción, el dataset se cargaría desde un servicio de almacenamiento (ej. S3), una API externa o una base de datos, evitando dependencias locales como Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [403]:
ruta = '/content/drive/MyDrive/Colab Notebooks/SentimentAPI/'
df =  pd.read_csv(f'{ruta}sentimentdataset_es.csv', encoding='latin1', sep=';', on_bad_lines='warn')

### <font size= 12 color="lightgreen" >Explorando el dataset <font>

In [404]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User',
       'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month',
       'Day', 'Hour'],
      dtype='object')

In [405]:
df.sample(3)

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
451,173,175,La impotencia se hunde a medida que los desafí...,Impotencia,20-06-2020 14:20,OverwhelmedHeart,Twitter,#Impotencia #Abrumado,8,15,Australia,2020,6,20,14
535,231,235,"Un viaje al pasado, hojeando las páginas de un...",Nostalgia,10-08-2018 15:45,DiaryExplorer,Facebook,#Nostalgia #ViajeDiario,12,25,Canadá,2018,8,10,15
195,473,477,"En el jardín de la alegría, cada flor susurra ...",Contentamiento,15-02-2022 12:20,GardenOfJoy,Facebook,#Alegría #PazInterior,25,50,Reino Unido,2022,2,15,12


### <font size=12 color=lightgreen> Crear dataframe filtrado </font>

In [420]:
#Renombrar columna Text por Texto
df.rename(columns={'Text': 'Texto'}, inplace=True)

# Crar df_filtrado con columnas Texto y Sentiment
df_filtrado = df[['Texto', 'Sentiment']]

# Eliminar registros nulos y datos nulos
df_filtrado.dropna(inplace=True)
df_filtrado = df_filtrado[df_filtrado['Texto'].notna()]

df_filtrado.sample(3)

,Texto,Sentiment
48,Asistió a un concierto de música clásica y sin...,Alegría
599,Practicando la atención plena con la meditación.,Positivo
50,Se embarcó en un viaje por carretera para volv...,Alegría


### <font size=12 color=lightgreen>Limpiar textos</font>

In [408]:
def pre_proccess_text(texto):
    # 1. Convertir a minúsculas
    texto = texto.lower()

    # 2. Normaliza el texto para separar las tildes de las letras
    texto = unicodedata.normalize('NFD', texto)

    # 3. Filtra y se queda solo con los caracteres que no son tildes
    texto = texto.encode('ascii', 'ignore').decode("utf-8")

    # 4. Eliminar URLs (http, https, www)
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)

    # 5. Eliminar hashtags (#Haghtag)
    # El patrón r'#\w+' busca el símbolo # seguido de caracteres alfanuméricos
    texto = re.sub(r'#\w+', '', texto)

    # 6. Eliminar menciones (@usuario)
    texto = re.sub(r'@\w+', '', texto)

    # 7. Eliminar caracteres especiales y números (opcional, según tu criterio)
    texto = re.sub(r'[^\w\s]', '', texto)

    # 8 Eliminar emojis
    texto = re.sub(r'[^\x00-\x7F]+', '', texto)


    # 8. Eliminar espacios extra
    texto = texto.strip()

    return texto



# Aplicar al DataFrame
df_filtrado['Texto_Limpio'] = df_filtrado['Texto'].apply(pre_proccess_text)

# Mostrar un ejemplo del antes y después
print(df_filtrado[['Texto', 'Texto_Limpio']].head())


                                               Texto  \
0  Abrumado por el peso del mundo, Atlas con los ...   
1  Abrumado por la cacofonía de las expectativas,...   
2  Abrumado por el laberinto de expectativas, un ...   
3  Abrumado por el apoyo recibido durante un desa...   
4  Llega el aburrimiento, el día se siente infini...   

                                        Texto_Limpio  
0  abrumado por el peso del mundo atlas con los h...  
1  abrumado por la cacofonia de las expectativas ...  
2  abrumado por el laberinto de expectativas un m...  
3  abrumado por el apoyo recibido durante un desa...  
4  llega el aburrimiento el dia se siente infinit...  


### <font size=12 color=lightgreen> Categorizar sentimientos </font>

In [411]:
# 1. Definimos las listas de sentimientos según su categoría


# Ver todos los sentimientos únicos para saber qué agrupar
print(df_filtrado['Sentiment'].unique())


['Abrumado' 'Aburrimiento' 'Aceptación' 'Admiración' 'Adoración'
 'Adrenalina' 'Afecto' 'Agotamiento' 'Agradecido' 'Agridulce'
 'Aislamiento' 'Alegría' 'Alegría al hornear' 'Alegría festiva'
 'Alegría juguetona' 'Alivio' 'Amabilidad' 'Amable' 'Amar' 'Amargura'
 'Ambivalencia' 'Amistad' 'Amor perdido' 'Angustia' 'Anhelo' 'Ánimo'
 'Ansiedad' 'Anticipación' 'Apreciación' 'Aprensivo' 'Armonía'
 'Arrepentimiento' 'Asco' 'Asombro' 'Aventura' 'Aventura Culinaria'
 'Bendición' 'Cálculo erróneo' 'Calma' 'Capricho' 'Cautivación'
 'Cazador de sueños' 'Celebración' 'Celos' 'Celoso' 'Chispa' 'Colorido'
 'Comodidad' 'Compasión' 'Compasivo' 'Compromiso' 'Conexión' 'Confiado'
 'Confianza' 'Confusión' 'Consciencia' 'Consuelo' 'Contemplación'
 'Contentamiento' 'Creatividad' 'Creatividad de la pasarela'
 'Cumplimiento' 'Curiosidad' 'De espíritu libre' 'Decepción' 'Desafío'
 'Desamor' 'Descubrimiento' 'Desesperación' 'Deslumbrar' 'Despectivo'
 'Determinación' 'Devastado' 'Disfrute' 'Diversión' 'Dolor' 'El

In [412]:
# 1. SENTIMIENTOS POSITIVOS (Bienestar, éxito, alegría)
positivos = [
    'Aceptacion', 'Admiracion', 'Adoracion', 'Adrenalina', 'Afecto', 'Agradecido',
    'Alegria', 'Alegria al hornear', 'Alegria festiva', 'Alegria juguetona', 'Alivio',
    'Amabilidad', 'Amable', 'Amar', 'Amistad', 'Animo', 'Apreciacion', 'Armonia',
    'Asombro', 'Aventura', 'Aventura culinaria', 'Bendicion', 'Calma', 'Capricho',
    'Cautivacion', 'Cazador de suenos', 'Celebracion', 'Chispa', 'Colorido', 'Comodidad',
    'Compasion', 'Compasivo', 'Compromiso', 'Conexion', 'Confiado', 'Confianza',
    'Consciencia', 'Consuelo', 'Contentamiento', 'Creatividad', 'Creatividad de la pasarela',
    'Cumplimiento', 'De espiritu libre', 'Descubrimiento', 'Deslumbrar', 'Determinacion',
    'Disfrute', 'Diversion', 'Elacion', 'Elegancia', 'Emocion', 'Emocionado', 'Empatico',
    'Empoderamiento', 'Encantamiento', 'Encanto', 'Energia', 'Entusiasmo', 'Esfuerzo renovado',
    'Esperanza', 'Euforia', 'Excitacion', 'Exito', 'Exploracion', 'Explosion artistica',
    'Extasis', 'Fascinante', 'Felicidad', 'Feliz', 'Grandeza', 'Gratitud', 'Hipnotico',
    'Iconico', 'Imaginacion', 'Inmersion', 'Inspiracion', 'Inspiracion creativa', 'Inspirado',
    'Intriga', 'Jugueton', 'La belleza de la naturaleza', 'La libertad del oceano', 'Libertad',
    'Lleno de alegria', 'Logro', 'Magia de invierno', 'Maravilla', 'Maravilla celestial',
    'Melodico', 'Motivacion', 'Optimismo', 'Orgullo', 'Orgulloso', 'Positividad', 'Positivo',
    'Reconfortante', 'Rejuvenecimiento', 'Resiliencia', 'Resplandor', 'Reunion alegre',
    'Reverencia', 'Romance', 'Satisfaccion', 'Serenidad', 'Tranquilidad', 'Triunfo',
    'Vibrancia', 'Viaje emocionante'
]


# 2. SENTIMIENTOS NEGATIVOS (Dolor, ira, miedo, estrés)
negativos = [
    'Abrumado', 'Aburrimiento', 'Agotamiento', 'Agridulce', 'Aislamiento', 'Amargura',
    'Amor perdido', 'Angustia', 'Anhelo', 'Ansiedad', 'Aprensivo', 'Arrepentimiento',
    'Asco', 'Celos', 'Celoso', 'Decepcion', 'Desafio', 'Desamor', 'Desesperacion',
    'Despectivo', 'Devastado', 'Dolor', 'Enojo', 'Entumecimiento', 'Envidia', 'Envidiar',
    'Envidioso', 'Frustracion', 'Frustrado', 'Impotencia', 'Intimidacion', 'Lastima',
    'Malo', 'Melancolia', 'Miedo', 'Negativo', 'Obstaculo', 'Odiar', 'Oscuridad', 'Pena',
    'Perdida', 'Presion', 'Resentimiento', 'Soledad', 'Sufrimiento', 'Temeroso', 'Temor',
    'Tormenta emocional', 'Traicion', 'Tristeza', 'Tristezaza', 'Verguenza'
]

# 3. SENTIMIENTOS NEUTRALES (O "Grises" que no definen éxito/fracaso)
# Aquí incluimos "Confuso" (Blender) y otros estados contemplativos
neutros = [
    'Ambivalencia', 'Anticipacion', 'Calculo erroneo', 'Confusion', 'Confuso',
    'Contemplacion', 'Curiosidad', 'Indiferencia', 'Neutral', 'Nostalgia',
    'Odisea culinaria', 'Pensive', 'Preguntarse', 'Reflexion', 'Restos',
    'Suspenso', 'Susurros del pasado', 'Travieso', 'Viaje', 'Viaje interior',
    'Visualizando la historia'
]

### <font color=lightgreen size=12>Función para categorizar sentimiento</font>

In [413]:
# 1. Función de categorización de sentimiento
def categorizar_sentimiento(sentimiento):
    # Limpiamos espacios en blanco y estandarizamos a título
    sent = str(sentimiento).strip().title()

    if sent in positivos:
        return 'Positivo'
    elif sent in negativos:
        return 'Negativo'
    else:
        # Por defecto, lo que no conocemos o es ambiguo va a Neutral para el MVP
        return 'Neutral'


In [414]:
# 2. Aplicamos la función a tu columna 'Sentimiento'
df_filtrado['Sentimiento_Final'] = df_filtrado['Sentiment'].apply(categorizar_sentimiento)

# 3. Verificamos cómo quedó la distribución
print(df_filtrado['Sentimiento_Final'].value_counts())

Sentimiento_Final
Neutral     330
Positivo    250
Negativo    152
Name: count, dtype: int64


### <font size=12 color=lightgreen> Exportar</font>

In [415]:
# Eliminar posibles nulos generados tras la limpieza
df_entrega = df_filtrado.dropna(subset=['Texto_Limpio'])
df_entrega = df_entrega[df_entrega['Texto_Limpio'].str.strip() != ""]

In [421]:
# Seleccionar solo lo necesario
df_entrega = df_entrega[['Texto_Limpio', 'Sentimiento_Final']]
df_entrega.sample(5)

,Texto_Limpio,Sentimiento_Final
160,compasion hacia los necesitados durante las va...,Neutral
405,completar con exito un proyecto de codificacio...,Positivo
234,decepcion desgarradora esperanzas destrozadas ...,Neutral
47,redescubri los dibujos animados de la infancia...,Neutral
67,pase horas perfeccionando un experimento de qu...,Neutral


In [422]:
# Exportar
df_entrega.to_csv(f'{ruta}dataset_listo_para_ML.csv', index=False)

print("Dataset exportado exitosamente.")

Dataset exportado exitosamente.


### <font size=12 color=lightgreen>Observación</font>

 Se realizó una normalización idiomática manual asistida por Excel para corregir traducciones y unificar el idioma del corpus, mejorando la coherencia semántica previa al entrenamiento del modelo.